# LancgChain / Chain - solution

### ![](../img/installation-ico.png) Installation

In [2]:
pip install langchain

  Using cached langchain-0.1.11-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
Using cached langchain-0.1.11-py3-none-any.whl (807 kB)
Using cached langchain_text_splitters-0.0.1-py3-none-any.whl (21 kB)
Note: you may need to restart the kernel to use updated packages.


### ![](../img/package-ico.png) Imports

In [5]:
from langchain_community.llms import Ollama
# pour le prompt de chat
from langchain_core.prompts import PromptTemplate
# pour le parser de sortie
from langchain_core.output_parsers import StrOutputParser


### ![](../img/parametrage-ico.png) Paramétrages

In [6]:
# notre modèle
llm = Ollama(model="mistral", temperature=0)

# notre prompt de chat
prompt = PromptTemplate.from_template(
    "Tell me a joke about {input}."
)
# notre parser de sortie
output_parser = StrOutputParser()

# notre chaîne d'exécution
chain = prompt | llm | output_parser

### ![](../img/jouer-ico.png) Executions

In [ ]:
# invocation simple
input = "clown"
chain.invoke({"input": input})


## Aller plus loin

### ![](../img/jouer-ico.png) Stream

In [ ]:
# invocation pour stream
for chunk in chain.stream({"input": input}):
    print(chunk, end="", flush=True)

### ![](../img/jouer-ico.png) ChatPrompt et JsonParser

In [7]:
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

mistral = llm

class Response(BaseModel):
    answer: str = Field(description="answer to the user's question")
    source: str = Field(description="source used to answer the user's question, should be a website")
    
json_parser = JsonOutputParser(pydantic_object=Response)

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=("answer the users question as best as possible.")),
        SystemMessage(content=(json_parser.get_format_instructions())),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

chain_chat = chat_prompt | mistral | json_parser

question = "What is the capital of Norway?"
# print(chat_prompt.format_messages(question=question))

chain_chat.invoke({"question": question})

{'answer': 'Oslo', 'source': 'https://en.wikipedia.org/wiki/Oslo'}